In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import duckdb
import pandas as pd
import glob
import time
import os
import matplotlib


In [3]:
#Creating a connection to the duck db database:
conn = duckdb.connect()

Location and loading of the dataset

In [4]:
os.chdir('/content/drive/MyDrive/Project ADSDB Max_Julian/landing/persistent')
#conn.execute("""SELECT * FROM read_csv_auto('owid-covid-data_v1_2023-09-25 16:24:46.csv')""").df()
owid_complete = conn.execute("""SELECT * FROM read_csv_auto(['owid-covid-data_v1_2023-09-25 16:24:46.csv'], union_by_name=true)""").df()

Displaying the dataset

In [5]:
owid_complete

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.50,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.50,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.50,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.50,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.50,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171626,LTU,Europe,Lithuania,2021-10-12,356226.0,1908.0,2157.429,5292.0,23.0,24.143,...,38.0,NaN,6.56,75.93,0.882,2750058.0,NaN,NaN,NaN,NaN
171627,LTU,Europe,Lithuania,2021-10-13,358901.0,2675.0,2156.286,5323.0,31.0,25.857,...,38.0,NaN,6.56,75.93,0.882,2750058.0,NaN,NaN,NaN,NaN
171628,LTU,Europe,Lithuania,2021-10-14,361937.0,3036.0,2261.714,5362.0,39.0,28.571,...,38.0,NaN,6.56,75.93,0.882,2750058.0,NaN,NaN,NaN,NaN
171629,LTU,Europe,Lithuania,2021-10-15,364929.0,2992.0,2342.571,5388.0,26.0,28.714,...,38.0,NaN,6.56,75.93,0.882,2750058.0,NaN,NaN,NaN,NaN


Displaying columns

In [6]:
owid_complete.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

We can have a look to the general statistics of our dataset



In [7]:
owid_complete.describe()

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
count,1.588620e+05,1.690210e+05,1.683910e+05,1.503370e+05,169028.000000,168403.000000,158862.000000,169021.000000,168391.000000,150337.000000,...,99789.000000,67900.000000,122875.000000,159492.000000,135048.000000,1.716310e+05,6244.000000,6244.000000,6244.000000,6244.000000
mean,5.960952e+06,9.543087e+03,9.578491e+03,6.239853e+04,68.256863,68.509051,103976.035962,155.642773,156.203972,875.834945,...,32.933269,48.815360,2.959946,73.682239,0.724925,1.095582e+08,31274.317579,9.508868,11.627348,1592.958056
std,3.280748e+07,1.125015e+05,9.020517e+04,2.747698e+05,500.935648,399.911758,156945.715710,1054.918606,556.706886,1073.633581,...,14.556587,33.581689,2.357946,7.643304,0.150914,4.825379e+08,70578.427987,12.059468,24.809548,1881.551627
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,7.700000,1.188000,0.300000,53.280000,0.397000,3.801000e+03,-37726.098000,-27.640000,-70.590000,-2752.924800
25%,8.974000e+03,0.000000e+00,8.570000e-01,1.340000e+02,0.000000,0.000000,2918.443000,0.000000,0.206000,59.704000,...,20.400000,16.603000,1.300000,69.590000,0.611000,5.931620e+05,172.800083,1.810000,-0.960000,100.311008
50%,7.061000e+04,4.000000e+00,3.200000e+01,1.024000e+03,0.000000,0.143000,28137.255500,0.594000,8.807000,402.205000,...,33.100000,48.901500,2.200000,75.090000,0.744000,5.579148e+06,6968.149850,7.660000,6.230000,1023.757515
75%,6.560088e+05,3.120000e+02,5.411430e+02,1.050400e+04,4.000000,6.143000,125306.999000,45.275000,96.368500,1381.782000,...,44.400000,83.741000,3.840000,79.930000,0.851000,2.267376e+07,34226.771500,14.342500,16.750000,2667.623600
max,4.255031e+08,7.213802e+06,6.109825e+06,2.901294e+06,43215.000000,8584.857000,737554.506000,193107.077000,27586.725000,5662.527000,...,76.100000,98.999000,13.050000,84.860000,0.955000,4.721383e+09,889935.300000,70.780000,377.630000,10292.916000


There are 67 columns in our dataset. To perform a better analysis, we are going to perform a selection of the best representative ones.

In [8]:
reduced_owid = owid_complete[['continent','date','location','total_cases','total_deaths','median_age',
                             'female_smokers','male_smokers','life_expectancy','total_vaccinations','new_vaccinations','people_vaccinated','human_development_index',
                             'population']]

In [9]:
reduced_owid

,continent,date,location,total_cases,total_deaths,median_age,female_smokers,male_smokers,life_expectancy,total_vaccinations,new_vaccinations,people_vaccinated,human_development_index,population
0,Asia,2020-01-03,Afghanistan,NaN,NaN,18.6,NaN,NaN,64.83,NaN,NaN,NaN,0.511,41128772.0
1,Asia,2020-01-04,Afghanistan,NaN,NaN,18.6,NaN,NaN,64.83,NaN,NaN,NaN,0.511,41128772.0
2,Asia,2020-01-05,Afghanistan,NaN,NaN,18.6,NaN,NaN,64.83,NaN,NaN,NaN,0.511,41128772.0
3,Asia,2020-01-06,Afghanistan,NaN,NaN,18.6,NaN,NaN,64.83,NaN,NaN,NaN,0.511,41128772.0
4,Asia,2020-01-07,Afghanistan,NaN,NaN,18.6,NaN,NaN,64.83,NaN,NaN,NaN,0.511,41128772.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171626,Europe,2021-10-12,Lithuania,356226.0,5292.0,43.5,21.3,38.0,75.93,3288110.0,5783.0,1771488.0,0.882,2750058.0
171627,Europe,2021-10-13,Lithuania,358901.0,5323.0,43.5,21.3,38.0,75.93,3294118.0,6008.0,1773689.0,0.882,2750058.0
171628,Europe,2021-10-14,Lithuania,361937.0,5362.0,43.5,21.3,38.0,75.93,3301415.0,7297.0,1776383.0,0.882,2750058.0
171629,Europe,2021-10-15,Lithuania,364929.0,5388.0,43.5,21.3,38.0,75.93,3309649.0,8234.0,1779615.0,0.882,2750058.0


We want to quantify the number of missing values in our dataset.

In [10]:
reduced_owid.isnull().sum() * 100 / len(reduced_owid)

continent                   3.956162
date                        0.000000
location                    0.000000
total_cases                 7.439798
total_deaths               12.406850
median_age                 18.941217
female_smokers             41.067173
male_smokers               41.858406
life_expectancy             7.072732
total_vaccinations         75.690872
new_vaccinations           80.065373
people_vaccinated          76.651071
human_development_index    21.314914
population                  0.000000
dtype: float64

We observe that columns related to vaccinations have >70% of missing values. Lets see if there is any relation between vaccination missing values and continents:

In [11]:
# Number of present values
continent_vaccination_present = reduced_owid.groupby(['continent'])['new_vaccinations'].count()
continent_vaccination_present

continent
Africa             770
Asia              8068
Europe           11996
North America     4027
Oceania            535
South America     3807
Name: new_vaccinations, dtype: int64

In [12]:
# Number of missing values
continent_vaccination_mis = reduced_owid.groupby(['continent'])['new_vaccinations'].apply(lambda x: x.isna().sum())
continent_vaccination_mis

continent
Africa           38612
Asia             23117
Europe           27956
North America    28565
Oceania           8971
South America     8417
Name: new_vaccinations, dtype: int64

As we want to merge both datasets, we have to adjust the timelapse of them. To do so, we will delete those rows which contain information about time earlier than 8/3/2022.

In [13]:
adjusted_owid = reduced_owid[reduced_owid['date']>'2022-03-08']
adjusted_owid

,continent,date,location,total_cases,total_deaths,median_age,female_smokers,male_smokers,life_expectancy,total_vaccinations,new_vaccinations,people_vaccinated,human_development_index,population
796,Asia,2022-03-09,Afghanistan,175353.0,7630.0,18.6,NaN,NaN,64.83,NaN,NaN,NaN,0.511,41128772.0
797,Asia,2022-03-10,Afghanistan,175525.0,7636.0,18.6,NaN,NaN,64.83,NaN,NaN,NaN,0.511,41128772.0
798,Asia,2022-03-11,Afghanistan,175872.0,7639.0,18.6,NaN,NaN,64.83,NaN,NaN,NaN,0.511,41128772.0
799,Asia,2022-03-12,Afghanistan,175960.0,7640.0,18.6,NaN,NaN,64.83,NaN,NaN,NaN,0.511,41128772.0
800,Asia,2022-03-13,Afghanistan,176062.0,7642.0,18.6,NaN,NaN,64.83,NaN,NaN,NaN,0.511,41128772.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170973,Europe,2023-09-17,Liechtenstein,21486.0,87.0,NaN,NaN,NaN,82.49,NaN,NaN,NaN,0.919,39355.0
170974,Europe,2023-09-18,Liechtenstein,21486.0,87.0,NaN,NaN,NaN,82.49,NaN,NaN,NaN,0.919,39355.0
170975,Europe,2023-09-19,Liechtenstein,21486.0,87.0,NaN,NaN,NaN,82.49,NaN,NaN,NaN,0.919,39355.0
170976,Europe,2023-09-20,Liechtenstein,21486.0,87.0,NaN,NaN,NaN,82.49,NaN,NaN,NaN,0.919,39355.0


Now we can create an Exploratory Data Analysis to see general characteristics of the data

Not able to read or load file. Please check your inputs and try again...


Imputing missing data

In [14]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

In [23]:
df = adjusted_owid
# Group the DataFrame by the 'Country' column
grouped = adjusted_owid.groupby(['continent'])

# Initialize an empty list to store the imputed DataFrames
imputed_dataframes = []

# Iterate through each group and impute missing values for all columns except 'Date'
for name, group in grouped:
    group.loc[:, group.columns != 'date'] = group.loc[:, group.columns != 'date'].interpolate()
    imputed_dataframes.append(group)

# Concatenate the imputed DataFrames back into one final DataFrame
imputed_df = pd.concat(imputed_dataframes, ignore_index=True)

<ipython-input-23-65b14300a057>:9: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in grouped:


In [24]:
imputed_df

,continent,date,location,total_cases,total_deaths,median_age,female_smokers,male_smokers,life_expectancy,total_vaccinations,new_vaccinations,people_vaccinated,human_development_index,population
0,Africa,2022-03-09,Algeria,265323.0,6858.0,29.1,0.7,30.4,76.88,1.370490e+07,NaN,7.461932e+06,0.748,44903228.0
1,Africa,2022-03-10,Algeria,265346.0,6860.0,29.1,0.7,30.4,76.88,1.370635e+07,NaN,7.470154e+06,0.748,44903228.0
2,Africa,2022-03-11,Algeria,265366.0,6861.0,29.1,0.7,30.4,76.88,1.370781e+07,NaN,7.478375e+06,0.748,44903228.0
3,Africa,2022-03-12,Algeria,265391.0,6861.0,29.1,0.7,30.4,76.88,1.370927e+07,NaN,7.486597e+06,0.748,44903228.0
4,Africa,2022-03-13,Algeria,265410.0,6862.0,29.1,0.7,30.4,76.88,1.371073e+07,NaN,7.494819e+06,0.748,44903228.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67970,South America,2023-09-17,Guyana,73385.0,1299.0,26.3,2.0,12.3,69.91,9.601330e+05,58394.0,4.952850e+05,0.682,808727.0
67971,South America,2023-09-18,Guyana,73385.0,1299.0,26.3,2.0,12.3,69.91,9.601330e+05,58394.0,4.952850e+05,0.682,808727.0
67972,South America,2023-09-19,Guyana,73385.0,1299.0,26.3,2.0,12.3,69.91,9.601330e+05,58394.0,4.952850e+05,0.682,808727.0
67973,South America,2023-09-20,Guyana,73385.0,1299.0,26.3,2.0,12.3,69.91,9.601330e+05,58394.0,4.952850e+05,0.682,808727.0


In [34]:
# We still have some missing values to impute
imputed_df.isnull().sum() * 100 / len(imputed_df)

# Imputing the remaining missing values
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
num_imp = imp_mean.fit_transform(imputed_df[['total_deaths','median_age', 'female_smokers', 'male_smokers', 'life_expectancy',
       'total_vaccinations', 'new_vaccinations', 'people_vaccinated',
       'human_development_index']])

# Saving them in the df
imputed_df[['total_deaths','median_age', 'female_smokers', 'male_smokers', 'life_expectancy',
       'total_vaccinations', 'new_vaccinations', 'people_vaccinated',
       'human_development_index']] = num_imp

**Exporting the preprocessed file**

In [37]:
imputed_df.to_csv('/content/drive/MyDrive/Project ADSDB Max_Julian/landing/persistent/owid_preprocessed_2023-10-21 17:57:36.csv')  # or True?